In this notebook, we will explore Netflix movies and TV shows with [kotlin/dataframe](https://github.com/Kotlin/dataframe). Also, we will use [lets-plot](https://github.com/JetBrains/lets-plot-kotlin) library for data visualization.

## Table of contents
* [**Imports**](attachment:./#Imports)
* [**Reading and first look**](attachment:./#Reading-and-first-look)
* [**TV Shows and Movies**](attachment:./#TV-Shows-and-Movies)
* [**Lifetimes and Release Times**](attachment:./#Lifetimes-and-Release-Times)
* [**Actors**](attachment:./#Actors)
* [**Countries**](attachment:./#Countries)
* [**Duration**](attachment:./#Duration)
* [**Ratings**](attachment:./#Ratings)

## Imports

We use the latest available versions of the libraries, the following line magic is responsible for this:

In [1]:
%useLatestDescriptors

Importing dataframe

In [1]:
%use dataframe(0.8.0-dev-724)

Importing the visualization library

In [2]:
%use lets-plot

We will also work with maps, so we need the corresponding libraries.

In [3]:
%use lets-plot-gt(gt="[23,)")
@file:DependsOn("org.geotools:gt-shapefile:[23,)")
@file:DependsOn("org.geotools:gt-cql:[23,)")

And write the necessary imports

In [4]:
import org.geotools.data.shapefile.ShapefileDataStoreFactory
import org.geotools.data.simple.SimpleFeatureCollection
import java.net.URL

## Reading and first look

To get started, need to read data from csv

In [5]:
val raw_df = DataFrame.read("netflix_titles.csv")

First look could be taken at its content

In [6]:
// taking a look at types and columns
raw_df.schema()

show_id: String
type: String
title: String
director: String?
cast: String?
country: String?
date_added: String?
release_year: Int
rating: String?
duration: String
listed_in: String
description: String


In [7]:
raw_df.size() // rowsCount x columnsCount

7787 x 12

In [8]:
raw_df.head() // return first five rows

DataFrame [5 x 12]

In [9]:
// Getting general statistics and info for each columns
raw_df.describe()

DataFrame [12 x 12]

Data consists of Netflix TV shows and movies up to 2020. Each row contains information about one specific project and consists of:
* `show_id` - unique show number
* `type` - ***TV Show*** or ***Movie***
* `title` - the name of a TV show or movie
* `director` - director's name
* `cast` - cast list
* `country` - the country where the title was released
* `date_added` - when the title was added on netflix
* `release_year` - the year the title was released
* `rating` - rating of the title
* `listed_in` - in which lists/genres the title is present on netflix
* `description` - title description

Before we get started, let's process the dataframe. It can be seen that `date_added` is of type `String`, let's [convert](https://kotlin.github.io/dataframe/convert.html) it to `LocalDate` for further convenience. Kotlin DataFrame provides built-in type converters for major types. We will use `String` -> `LocalDate` conversion and specify date format [pattern](https://docs.oracle.com/javase/8/docs/api/java/time/format/DateTimeFormatter.html)

In [10]:
val df = raw_df.dropNulls { date_added } // remove rows where `date_added` is not specified
        .convert { date_added }.toLocalDate("MMMM d, yyyy") // convert date_added to LocalDate using date pattern
// let's look at what type of column it turned out
df.date_added.type() 

kotlinx.datetime.LocalDate

## TV Shows and Movies

First, let's see what more shows or films.

In [11]:
ggplot(raw_df.toMap()) +
    geomBar() { x="type"; fill="type" } +
    scaleFillManual(listOf("#00BCD4", "#009688")) +
    ggtitle("Count of TV Shows and Movies") +
    ggsize(900, 550)

It can be seen that the number of films on Netflix is about twice the number of TV shows. But has it always been this way? To do this, let's see if there was such a year when the number of TV Shows was more than Movies and let's see the cumulative amount for Movies and TV Shows.

In [12]:
val df_date_count = df.convert { date_added }.with { it.year } // converting `date_added` to extract `year`
  .groupBy { date_added } // grouping by `year` stored in `date_added`
  .aggregate { 
      count { type == "TV Show" } into "tvshows" // counting TV Shows into column `tvshows`
      count { type == "Movie" } into "movies" // counting Movies into column `movies`
  }
df_date_count

DataFrame [14 x 3]

Let's hold on and see how we can simplify this expression using more advanced operations. First of all, we can combine conversion of `date_added` into `year` and grouping using [`map`](https://kotlin.github.io/dataframe/map.html) function within [column selector](https://kotlin.github.io/dataframe/columnselectors.html).

In [13]:
val df_date_count = df.groupBy { date_added.map { it.year } } // grouping by year added extracted from `date_added`
  .aggregate { 
      count { type == "TV Show" } into "tvshows" // counting TV Shows into column `tvshows`
      count { type == "Movie" } into "movies" // counting Movies into column `movies`
  }

Our [groupBy aggregation](https://kotlin.github.io/dataframe/groupby.html#aggregation) adds new columns for "TV Show" and "Movie". This is exactly what [`pivot`](https://kotlin.github.io/dataframe/pivot.html) does: generates new columns for every unique value in `type`.

In [14]:
df.groupBy { date_added.map { it.year } }
  .pivot { type }

PivotGroupBy: 14 x 2

After `type` column is pivoted, we call [`aggregate`](https://kotlin.github.io/dataframe/pivot.html#aggregation) to specify metrics to be calculated for every data group.

In [13]:
df.groupBy { date_added.map { it.year } }
  .pivot { type }.aggregate { count () }

DataFrame [14 x 2]

Simple statistics can be aggregated without `aggregate`:

In [17]:
df.groupBy { date_added.map { it.year } }
  .pivot { type }.count()

DataFrame [14 x 2]

For `count` statistics there is even shorter API [`pivotCounts`](https://kotlin.github.io/dataframe/pivot.html#pivotcounts).

Here is the final version:

In [19]:
val df_date_count = df.groupBy { date_added.map { it.year } }.pivotCounts { type }
df_date_count

DataFrame [14 x 2]

Now we will prepare dataframe for rendering. We will call [`flatten`](https://kotlin.github.io/dataframe/flatten.html) to remove column grouping and convert dataframe to `Map`.

In [20]:
val data = df_date_count.flatten().toMap()

ggplot(data) +
    geomArea(color = "#BF360C", fill = "#BF360C", alpha = 0.5) { x="date_added"; y="TV Show" } +
    geomArea(color = "#01579B", fill = "#01579B", alpha = 0.5) { x="date_added"; y="Movie" } +
    theme(
        panelBackground = elementRect(color="#ECEFF1", fill = "#ECEFF1"),
        panelGrid = elementBlank(),
    ) +
    xlab("year") +
    ylab("count") +
    ggtitle("Number of titles by year") +
    ggsize(800, 500)

It can be seen that more films were added every year than shows. Obviously, the cumulative sum of the movies was also always higher than the TV Shows, but let's build such a plot.

In [21]:
val df_cumsum_titles = df_date_count
    .sortBy { date_added } // sorting by date_added
    .cumSum { type.all() } // count cumulative sum for columns `TV Show` and `Movie` that are nested under column `type`
df_cumsum_titles

DataFrame [14 x 2]

In [22]:
ggplot(df_cumsum_titles.flatten().toMap()) +
    geomArea(color = "#BF360C", fill = "#BF360C", alpha = 0.5) { x="date_added"; y="TV Show" } +
    geomArea(color = "#01579B", fill = "#01579B", alpha = 0.5) { x="date_added"; y="Movie" } +
    theme(
        panelBackground = elementRect(color="#ECEFF1", fill = "#ECEFF1"),
        panelGrid = elementBlank(),
    ) +
    xlab("year") +
    ylab("cumulative count") +
    ggtitle("Cumulative count of titles by year") +
    ggsize(800, 500)

## Lifetimes and Release Times

Let's take a look at the distribution by the lifetime of titles on the platform. To do this, find the most recently uploaded title and calculate the difference between the date it was added and the maximum date found.

In [23]:
import kotlinx.datetime.*

In [24]:
val maxDate = df.date_added.max()
val df_days = df.add {
    "days_on_platform" from { date_added.daysUntil(maxDate) } // adding column for number of days on the platform
    "months_on_platform" from { date_added.monthsUntil(maxDate) } // adding column for number of months on the platform
    "years_on_platform" from { date_added.yearsUntil(maxDate) } // // adding column for number of years on the platform
}

In [25]:
val plot = ggplot(df_days["type", "days_on_platform"].toMap())
val p1 = plot + 
        geomHistogram(color="#ECEFF1", fill="#ef0b0b") { x="days_on_platform"; y="..density.." } +
        geomDensity(alpha=0.5, fill="#0befef") { x="days_on_platform" } +
        xlab("days") + ggtitle("Age distribution (in days) on Netflix")
val p2 = plot +
        geomBoxplot() { x="type"; y="days_on_platform"; fill="type" } +
        scaleFillManual(listOf("#792020", "#207979")) +
        ylab("days") + ggtitle("Boxplot for age (in days) by type")

GGBunch()
    .addPlot(p1, 0, 0, 500, 450)
    .addPlot(p2, 500, 0, 500, 450)

The age distribution of titles on the platform is similar to movies and TV shows. But you can see in the second graph that there are very old titles among the movies compared to the shows. Let's take a closer look at this moment. To do this, let's build a graph of the duration in years of being on the platform of films and shows.

In [26]:
ggplot(df_days.select { type and years_on_platform }.toMap()) + // selecting two columns
    geomBar(position = Pos.dodge) { x = "years_on_platform"; fill = "type" } +
    scaleFillManual(listOf("#bc3076", "#30bc76")) +
    xlab("years") +
    ggtitle("Years of Movies and TV Shows on Netflix") +
    ggsize(900, 500)

As you can see, movies are usually older than TV shows.
After that, you might ask yourself: how quickly were titles added to Netflix after their release? Well, finding the answer to it will be quite simple.

In [27]:
val df_years = df
                // adding a new column of the difference between the year of release and the year of addition
                .add("years_off_platform") {
                    date_added.year - release_year
                }
                // dropping negative values and equal to zero
                .filter { "years_off_platform"<Int>() > 0 }

We dropped negative values because it happens that titles are added to the platform while it is still in production. Also dropped the zero values as they are of no interest. 

In [28]:
ggplot(df_years.select { years_off_platform }.toMap()) + 
    geomPoint(stat = Stat.count(), size = 7.5) { x="years_off_platform"; color="years_off_platform" } +
    scaleColorGradient(low = "#97a6d0", high = "#00256e") +
    theme().legendPosition(0.9, 0.83) +
    xlab("years") +
    ggtitle("How long does it take for a title to be added to Netflix?") +
    ggsize(1000, 500)

Well, let's build the informal top charts for the oldest and newest movies and TV shows.

* ***Top 5 movies with the most days on Netflix***

In [29]:
// Top 5 oldest movies
df_days
    .filter { type == "Movie" } // filtering by type
    .sortByDesc { days_on_platform } // sorting by number of days on Netflix
    .select { type and title and country and date_added and release_year and duration } // selecting required columns
    .head() // taking first five rows

DataFrame [5 x 6]

* ***Top 5 movies recently added on Netflix***

In [29]:
// Top 5 newest movies
df_days
    .filter { type == "Movie" }
    .sortBy { days_on_platform }
    .select { type and title and country and date_added and release_year and duration }
    .head()

DataFrame [5 x 6]

* ***Top 5 TV Shows with most days on Netflix***

In [30]:
// Top 5 oldest shows
df_days
    .filter { type == "TV Show" }
    .sortByDesc { days_on_platform }
    .select { type and title and country and date_added and release_year and duration }
    .head()

DataFrame [5 x 6]

* ***Top 5 TV Shows recently added on Netflix***

In [31]:
// Top 5 newest shows
df_days
    .filter { type == "TV Show" }
    .sortBy { days_on_platform }
    .select { type and title and country and date_added and release_year and duration }
    .head()

DataFrame [5 x 6]

You might be interested in what months are titles added most often?

In [32]:
val df_split_date = df
                    // splitting dates into four columns
                    .split { date_added }.by { listOf(it, it.dayOfWeek, it.month, it.year) }
                    .into("date", "day", "month", "year")
                    .sortBy("month") // sorting by month

In [33]:
ggplot(
    df_split_date
        .groupBy { year and month }.count() // counting how many titles were added by year and month
        .convert { month }.toStr() // converting month to String for rendering
        .toMap()
    ) + 
    geomTile(height = 0.9, width = 0.9) { x = "year"; y="month"; fill="count" } +
    theme(panelBackground = elementBlank(),panelGrid = elementBlank()) +
    scaleFillGradient(low = "#FFF3E0", high = "#E65100") +
    ggtitle("Content additions by month and year") +
    ggsize(900, 700)

## Actors

In this section, let's take a look at the actors and directors who make the content. First, let's determine the average number of actors in titles.

In [34]:
// splitting cast and couting number of actors
val cast_df = df
    .split { cast }.by(',').inplace()
    .add("size_cast") { "cast"<List<String>>().size }

In [35]:
ggplot(cast_df.toMap()) +
    geomHistogram(bins = 50, showLegend = false) { x = "size_cast"; fill="..count.." } +
    scaleFillGradient(low = "#E0F7FA", high = "#006064") +
    xlab("actors") +
    ggtitle("Number of people on cast") +
    ggsize(950, 650)

It can be seen that usually 8-9 people are included in the cast.

But what about who exactly is involved in creating the content? Let's take a look at these actors and how many times they took part in movies and shows.

In [36]:
// counting the participation of each actor
val actors_df = cast_df.cast.explode().valueCounts()
actors_df

... showing only top 20 of 32847 rows DataFrame [32847 x 2]

In [37]:
ggplot(actors_df.toMap()) +
    geomHistogram(
        stat = Stat.identity, sampling = samplingPick(30), color = "#E8F5E9", boundary = 1.0, showLegend = false
    ) { x="cast"; y="count"; fill="cast" } +
    coordFlip() +
    scaleFillHue() +
    xlab("cast") +
    ggtitle("Top 30 actors") +
    ggsize(950, 900)

Anupam Kher is definitely in the lead with 42 titles. Now we will split the castes for participation in movies or shows.

In [38]:
val actors = cast_df.pivot { type }.aggregate {
    cast.explode().valueCounts()
}
actors

DataRow [2]

In [39]:
val p1 = ggplot(actors.`TV Show`.toMap()) +
            geomHistogram(
              stat = Stat.identity, sampling = samplingPick(30), color = "#E8F5E9", boundary = 1.0, showLegend = false
            ) { x="cast"; y="count"; fill="cast" } +
             coordFlip() +
            scaleFillGradient(low = "#263238", high = "#ECEFF1") +
            xlab("cast") +
            ggtitle("Top 30 actors in Shows")

val p2 = ggplot(actors.Movie.toMap()) +
            geomHistogram(
              stat = Stat.identity, sampling = samplingPick(30), color = "#E8F5E9", boundary = 1.0, showLegend = false
            ) { x="cast"; y="count"; fill="cast" } +
             coordFlip() +
            scaleFillGradient(low = "#006064", high = "#E0F7FA") +
            xlab("cast") +
            ggtitle("Top 30 actors in Movies")

GGBunch()
    .addPlot(p1, 0, 0, 500, 700)
    .addPlot(p2, 500, 0, 500, 700)

How about directors? Let's see the top 10 directors with more appearence on Netflix catalog.

In [40]:
val directors_df = df.valueCounts { director }

In [41]:
ggplot(directors_df.toMap()) +
    geomHistogram(
            stat = Stat.identity, sampling = samplingPick(10), color = "#E8F5E9", boundary = 1.0, showLegend = false
        ) { x = "director"; y = "count"; fill = "director"} +
        scaleFillHue() +
        coordFlip() +
        xlab("Name") +
        ggtitle("Top 10 directors") +
        ggsize(850, 500)

These people work very productively.

## Countries

Here we will examine the distribution of content by country.

We need to do some small manipulations to render map.

In [42]:
val factory = ShapefileDataStoreFactory()

In [43]:
val worldFeatures : SimpleFeatureCollection = with("naturalearth_lowres") {
    val url = "https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/shp/${this}/${this}.shp"
    factory.createDataStore(URL(url)).featureSource.features
}

// Convert Feature Collection to SpatialDataset.
// Use 10 decimals to encode floating point numbers (this is the default).
val world = worldFeatures.toSpatialDataset(10)
val voidTheme = theme().axisLineBlank().axisTextBlank().axisTicksBlank().axisTitleBlank()
val worldLimits = coordMap(ylim = -55 to 85)

Let's add another dataframe with country labels.

In [44]:
val countries = DataFrame.readCSV("country_codes.csv")
countries.head()

DataFrame [5 x 5]

In [45]:
// counting number of titles by county and joining them with country codes dataframe
val df_country = df.valueCounts { country }.join(countries)
df_country

... showing only top 20 of 68 rows DataFrame [68 x 6]

In [46]:
ggplot() +
    geomMap(data = df_country.toMap(), map = world, mapJoin = "iso" to "iso_a3", color = "white") { fill="count" } +
    scaleFillGradient(low = "#FFF3E0", high = "#E65100", name = "Number of Titles") +
    ggsize(1000, 800) + voidTheme + worldLimits

The map clearly shows where the content is mainly produced and gets to Netflix. Let's take a closer look at the top of such countries.

In [47]:
ggplot(df_country[0..9].sortByDesc { count }.toMap()) +
    geomBar(stat = Stat.identity, fill = "#00796B") { x="country"; y="count" } +
    ggtitle("Top 10 Countries") +
    ggsize(900, 450)

## Duration

How long does the content usually last to keep the viewer?

In [48]:
val df_dur = df
                .split { duration }.by(" ").inward("duration_num", "duration_scale") // splitting duration by time and scale inward
                .convert { "duration"["duration_num"] }.toInt() // converting by column path
                .update { "duration"["duration_scale"] }.with { if (it == "Seasons") "Season" else it }
df_dur.head()

DataFrame [5 x 12]

In [49]:
val durations = df_dur.pivot { type }.values { duration }
durations

DataRow [2]

In [50]:
val p1 = ggplot(durations.Movie.toMap()) { x="duration_num" } + 
            geomHistogram(bins = 100, color="#e0f7fa", fill="#00BCD4") { y="..density.." } +
            geomDensity(alpha=1, size =1, color = "#d41900") + xlab("minutes") +
            ggtitle("Distribution of movies duration in minutes")

val p2 = ggplot(durations.`TV Show`.toMap()) { x = "duration_num" } + statCount(fill = "#00BCD4") + xlab("seasons") + 
            ggtitle("Number of seasons of tv show")

GGBunch()
    .addPlot(p1, 0, 0, 1000, 500)
    .addPlot(p2, 0, 500, 1000, 500)

And according to tradition, the top longest movies and TV shows.

* ***Top 5 movies with highest duration***

In [51]:
df_dur.xs("Movie") { type }
    .sortByDesc { duration.duration_num }.head()
    .select { title and country and date_added and release_year and duration.all() }

DataFrame [5 x 6]

* ***Top 5 TV shows with most seasons***

In [52]:
df_dur.xs("TV Show") { type }
    .sortByDesc { duration.duration_num }.head()
    .select { title and country and date_added and release_year and duration.all() }

DataFrame [5 x 6]

And in the top content producing countries, how long are movies and TV shows?

In [53]:
val list_top_countries = df_country.country.take(10).toSet()

val df_cntr = df_dur
    .filter { country in list_top_countries }
    .pivot { type }.aggregate { 
        groupBy{ country }.mean { duration.duration_num }
    }
df_cntr

DataRow [2]

In [54]:
val p1 = ggplot(df_cntr.Movie.toMap()) +
        geomHistogram(
            stat = Stat.identity, boundary = 1.0, color = 0xE0F7FA, showLegend = false
        ) { 
            x = "country";
            y = "duration_num";
            fill = "duration_num";
        } +
        scaleFillGradient(low = "#ECEFF1", high = "#263238") +
        ylab("Minute") +
        xlab("Name") +
        ggtitle("Top 30 cast on Movies")
        
val p2 = ggplot(df_cntr.`TV Show`.toMap()) +
        geomHistogram(
            stat = Stat.identity, boundary = 1.0, color = 0xE0F7FA, showLegend = false
        ) { 
            x = "country";
            y = "duration_num";
            fill = "duration_num"
        } +
        scaleFillGradient(low = "#E0F7FA", high = "#006064") +
        ylab("Season") +
        xlab("Name") +
        ggtitle("Top 30 cast on TV Shows")

GGBunch()
    .addPlot(p1, 0, 0, 900, 550)
    .addPlot(p2, 0, 550, 900, 550)

## Ratings

Finally, let's take a look at the rating column.
Here we will find out what is the most commonly assigned rating for films and shows.

In [55]:
ggplot(df.toMap()) +
        geomHistogram(boundary = 1.0, color = 0xE0F7FA, showLegend = false) { x = "rating"; fill="rating" } +
        scaleFillHue() +
        xlab("Rating") +
        ggtitle("Rating of Titles") +
        ggsize(950, 500)

In [56]:
ggplot(df.toMap()) + 
    geomBar(position=Pos.dodge) {x="rating"; fill="type"} + 
    scaleFillManual(listOf("#607D8B", "#00BCD4")) +
    xlab("Rating") +
    ggtitle("Rating of Titles") +
    ggsize(950, 500)